# Library

In [50]:
# Librerie standard
import os
import random
import time
import re
from pathlib import Path
from collections import defaultdict, Counter
from itertools import islice

# Librerie per il trattamento delle immagini
import cv2
import imageio.v3 as imageio
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from torchvision.transforms import functional as TF
import torchvision.transforms as transforms

# Librerie per il machine learning e deep learning
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as func
import torchvision.models as models
from sklearn.svm import SVC

# Librerie per la gestione dei dati
import pandas as pd
import json
import orjson
import shutil 

# Librerie per il parallelismo e il multiprocessing
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

# Librerie per il progresso e il monitoraggio
from tqdm import tqdm

# Librerie per la gestione dei dataset
from torch.utils.data import Dataset, DataLoader

# Librerie per modelli e trasformazioni in PyTorch
from torchvision import transforms

from collections import Counter
from sklearn.model_selection import train_test_split
import warnings

from torchvision import models
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch import optim
import torch
import json
import random
from collections import defaultdict
import ast

# Path

In [51]:
#Output folders and file names
OUT_COCO_JSON_NM = 'COCO_annotations_new.json'
OUT_IMAGE_FLDR_NM = 'images'
OUT_CFG_FLDR_NM = 'YOLO_cfg'
RANDOM_SEED = 2023

in_dataset_pth = Path('/kaggle/input/our-xview-dataset')
out_dataset_pth = Path('/kaggle/working/')
img_fldr = Path(f'/kaggle/input/our-xview-dataset/{OUT_IMAGE_FLDR_NM}')
cfg_fldr_pth = Path(f'/kaggle/input/our-xview-dataset/{OUT_CFG_FLDR_NM}')

coco_json_pth = in_dataset_pth / OUT_COCO_JSON_NM
train_txt_pth = cfg_fldr_pth / 'train.txt'
val_txt_pth = cfg_fldr_pth / 'val.txt'
test_txt_pth = cfg_fldr_pth / 'test.txt'

#DATASET
train_path = '/kaggle/working/train.json'
test_path = '/kaggle/working/test.json'
val_path = '/kaggle/working/val.json'

random.seed(RANDOM_SEED)

In [52]:
# Pulizia dell'output per cartelle specifiche
def clean_output(output_dir):
    if output_dir.exists() and output_dir.is_dir():
        for item in output_dir.iterdir():
            if item.is_dir():
                shutil.rmtree(item)  # Rimuove la sotto-cartella
            else:
                item.unlink()  # Rimuove il file
        print(f"Cartella {output_dir} pulita.")
    else:
        print(f"Cartella {output_dir} non trovata. Nessuna azione necessaria.")

# Pulisce la cartella di output prima di avviare il processo
clean_output(out_dataset_pth)

Cartella /kaggle/working pulita.


In [53]:
# Sopprime i warning specifici del modulo skimage
warnings.filterwarnings("ignore", 
    message="Applying `local_binary_pattern` to floating-point images may give unexpected results.*")

# Splitting

In [54]:
def split_stratified(json_file, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    # Carica il JSON
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Raggruppare le annotazioni per category_id
    category_images = defaultdict(list)
    for annotation in data['annotations']:
        category_id = annotation['category_id']
        image_id = annotation['image_id']
        category_images[category_id].append(image_id)
    
    # Genera gli split per ogni category_id
    train_images, val_images, test_images = set(), set(), set()
    for category_id, image_ids in category_images.items():
        # Mescola gli image_id
        random.shuffle(image_ids)
        
        # Calcola i limiti per train, validation, e test
        total = len(image_ids)
        train_end = int(total * train_ratio)
        val_end = int(total * (train_ratio + val_ratio))
        
        # Aggiungi agli split
        train_images.update(image_ids[:train_end])
        val_images.update(image_ids[train_end:val_end])
        test_images.update(image_ids[val_end:])
    
    # Filtra le immagini e annotazioni per ciascuno split
    def filter_data(split_images):
        filtered_images = [image for image in data['images'] if image['id'] in split_images]
        filtered_annotations = [annotation for annotation in data['annotations'] if annotation['image_id'] in split_images]
        return {'images': filtered_images, 'annotations': filtered_annotations, 'categories': data['categories']}
    
    # Crea i nuovi JSON per train, validation, e test
    train_data = filter_data(train_images)
    val_data = filter_data(val_images)
    test_data = filter_data(test_images)
    
    # Salva i file JSON
    with open('train.json', 'w') as f:
        json.dump(train_data, f, indent=4)
    
    with open('val.json', 'w') as f:
        json.dump(val_data, f, indent=4)
    
    with open('test.json', 'w') as f:
        json.dump(test_data, f, indent=4)

In [55]:
# Chiamata della funzione
split_stratified(coco_json_pth)

# DataLoader

In [192]:
import os
import json
import torch
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
import ast

class CustomDataset(Dataset):
    def __init__(self, coco_json_file, img_dir, aug=False):
        """
        Inizializza il dataset personalizzato.
        Args:
        - coco_json_file: Il file JSON contenente le annotazioni.
        - img_dir: La cartella delle immagini.
        - aug: Booleano per attivare o meno l'augmentazione.
        """
        def generate_id(file_name):
            return file_name.replace('_', '').replace('.jpg', '').replace('img', '')
        
        # Carica il file JSON delle annotazioni
        with open(coco_json_file, 'r') as f:
            coco_data = json.load(f)
        
        # Crea una struttura per le annotazioni
        self.image_annotations = {}
        self.image_bboxes = {}
        
        # Estrai le classi (categorie) dal file JSON
        self.classes = {}
        for category in coco_data['categories']:
            for key, value in category.items():
                self.classes[int(key)] = value  # Associa l'ID categoria al nome
        
        # Aggiungi la mappa di annotazioni
        for annotation in coco_data['annotations']:
            image_id = annotation['image_id']
            category_id = annotation['category_id']
            bbox = annotation['bbox']  # Formato COCO [x_min, y_min, width, height]
            
            if image_id not in self.image_annotations:
                self.image_annotations[image_id] = []
                self.image_bboxes[image_id] = []
            
            self.image_annotations[image_id].append(category_id)
            self.image_bboxes[image_id].append(bbox)
        
        # Mappa per associare ID immagine a file_name
        self.image_info = {
            int(generate_id(image['file_name'])): image['file_name']
            for image in coco_data['images']
        }
        
        # Salva i percorsi delle immagini nel formato richiesto
        self.img_dir = img_dir
        self.image_paths = [
            os.path.join(img_dir, image['file_name'])
            for image in coco_data['images']
        ]
        
        # Trasformazioni di base e di augmentation
        self.base_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),   
        ])
        
        self.aug_transform = transforms.Compose([
            transforms.Resize((320, 320)),
            transforms.ToTensor(),
        ]) 
        
        self.aug = aug
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        # Estrai il nome dell'immagine e l'ID corrispondente
        img_path = self.image_paths[index]
        img_name = os.path.basename(img_path)
        img_id = int(img_name.replace('_', '').replace('.jpg', '').replace('img', ''))
        
        if img_id not in self.image_info:
            raise ValueError(f"Immagine {img_name} non trovata nel file COCO")
        
        if not os.path.exists(img_path):
            raise ValueError(f"Immagine non trovata nel percorso: {img_path}")
        
        # Carica l'immagine
        image = Image.open(img_path).convert('RGB')
        original_width, original_height = image.size
        
        # Applica le trasformazioni
        if self.aug:
            image_tensor = self.aug_transform(image)
        else:
            image_tensor = self.base_transform(image)
        
        # Estrai le annotazioni e i bounding boxes
        categories = self.image_annotations.get(img_id, [])
        bboxes = self.image_bboxes.get(img_id, [])
        
        if not bboxes:  # Immagini senza annotazioni
            target = {
                "boxes": torch.zeros((0, 4), dtype=torch.float32),
                "labels": torch.zeros((0,), dtype=torch.int64)
            }
        else:
            # Converte da formato COCO [x_min, y_min, width, height] a [x_min, y_min, x_max, y_max]
            scale_x = 320 / original_width
            scale_y = 320 / original_height
            
            # Scaling dei bounding boxes
            scaled_bboxes = []
            for bbox_str in bboxes:
                bbox = ast.literal_eval(bbox_str)  # Converte la stringa in una lista
                x_min, y_min, width, height = bbox
                x_max = x_min + width
                y_max = y_min + height
                
                # Verifica che x_min < x_max e y_min < y_max
                if x_min >= x_max or y_min >= y_max:
                    # Se il bounding box non è valido, salta questa immagine
                    return None, None
                
                scaled_bboxes.append(torch.tensor([  
                    float(x_min) * scale_x,               # x_min
                    float(y_min) * scale_y,               # y_min
                    float(x_max) * scale_x,               # x_max
                    float(y_max) * scale_y                # y_max
                ], dtype=torch.float32))
            
            target = {
                "boxes": torch.stack(scaled_bboxes),
                "labels": torch.tensor(categories, dtype=torch.int64)
            }
        
        return image_tensor, target

In [193]:
def collate_fn(batch):
    """
    Funzione di collation per il DataLoader, utile per il batching di immagini e annotazioni.
    La funzione restituirà un batch di immagini e un batch di target, formattato correttamente per Faster R-CNN.
    
    Args:
    - batch: lista di tuple (image, target)
    
    Returns:
    - images: batch di immagini
    - targets: lista di dizionari contenenti le annotazioni per ogni immagine
    """
    # Separa immagini e target
    images, targets = zip(*batch)

    # Converte la lista di immagini in un batch di immagini
    images = list(images)

    # Restituisci il batch
    return images, list(targets)

In [194]:
# Creazione dei dataset
train_dataset = CustomDataset(train_path, img_fldr,  aug=True)
valid_dataset = CustomDataset(val_path, img_fldr, aug=False)  
test_dataset = CustomDataset(test_path, img_fldr, aug=False)  

# Creazione dei DataLoader
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(valid_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

## Check DataLoader

In [195]:
# Numero totale di campioni per ogni DataLoader
train_size = len(train_loader.dataset)
val_size = len(val_loader.dataset)
test_size = len(test_loader.dataset)

# Numero di batch per ogni DataLoader
train_batches = len(train_loader)
val_batches = len(val_loader)
test_batches = len(test_loader)

# Visualizza i risultati
print(f"Numero totale di elementi nel train_loader: {train_size}")
print(f"Numero totale di batch nel train_loader: {train_batches}")
print(f"Numero totale di elementi nel val_loader: {val_size}")
print(f"Numero totale di batch nel val_loader: {val_batches}")
print(f"Numero totale di elementi nel test_loader: {test_size}")
print(f"Numero totale di batch nel test_loader: {test_batches}")

# Somma totale degli elementi nei DataLoader
total_elements = train_size + val_size + test_size
print(f"Numero totale di elementi in tutti i DataLoader: {total_elements}")

Numero totale di elementi nel train_loader: 31248
Numero totale di batch nel train_loader: 15624
Numero totale di elementi nel val_loader: 19664
Numero totale di batch nel val_loader: 9832
Numero totale di elementi nel test_loader: 19803
Numero totale di batch nel test_loader: 9902
Numero totale di elementi in tutti i DataLoader: 70715


# Modello Faster R-CNN (Resnet50)

In [201]:
def compute_class_weights(dataset):
    # Conta la frequenza di ogni classe nel dataset
    class_counts = np.zeros(len(dataset.classes))  # Non consideriamo lo sfondo, quindi senza +1
    
    # Usa tqdm per monitorare il progresso mentre si itera sul dataset
    for _, targets in tqdm(dataset, desc="Calcolo frequenze delle classi", leave=False):
        # Controlla se targets è None
        if targets is None:
            continue
        
        for target in targets['labels']:  # Assumendo che 'labels' contenga le etichette delle classi
            if target != 0:  # Ignora lo sfondo nelle etichette
                class_counts[target] += 1

    # Calcola i pesi per le classi
    total_count = sum(class_counts)  # Totale delle etichette (senza sfondo)
    
    # Inizializza il peso per lo sfondo
    background_weight = 0.2  # Assegna un peso fisso per lo sfondo (sfondo è il 20% del dataset)
    class_counts_with_background = np.copy(class_counts)
    class_counts_with_background = np.append(class_counts_with_background, 0)  # Aggiungi lo sfondo
    class_counts_with_background[0] = background_weight * total_count  # Aggiorna il conteggio dello sfondo

    # Calcola i pesi inversamente proporzionali alla frequenza
    # Evita divisione per zero, mettendo i pesi delle classi con count zero a un valore molto alto
    class_weights = np.divide(total_count, class_counts_with_background, where=class_counts_with_background != 0)

    return class_weights

In [208]:
def train_and_validate(model, train_loader, val_loader, optimizer, device, class_weights, num_epochs=10, save_model=True):
    """
    Funzione per il training e la validazione del modello Faster R-CNN con pesi delle classi.
    """
    model.to(device)
    losses_per_epoch = []

    # Assicurati che i pesi siano un tensor PyTorch
    class_weights = torch.tensor(class_weights).to(device)

    for epoch in range(num_epochs):
        print(f"\nEpoca {epoch + 1}/{num_epochs}")
        model.train()
        total_loss = 0

        # Training loop con tqdm
        train_loop = tqdm(train_loader, desc="Training", leave=False)
        for images, targets in train_loop:
            # Controlla se le immagini sono valide (non None)
            if images is None or targets is None:
                continue
            
            images = [img.to(device) if img is not None else None for img in images]
            targets = [{k: v.to(device) if v is not None else None for k, v in t.items()} for t in targets]

            # Calcola le perdite
            loss_dict = model(images, targets)
            
            # Pondera le perdite per le classi
            # In questo caso, moltiplichiamo solo i contributi alla loss delle classi
            losses = loss_dict['loss_classifier'] + loss_dict['loss_box_reg'] + loss_dict['loss_objectness'] + loss_dict['loss_rpn_box_reg']

            # Se ci sono altre perdite legate alle classi, ponderale separatamente
            for key in loss_dict:
                if 'loss' in key and key != 'loss_classifier' and key != 'loss_box_reg' and key != 'loss_objectness' and key != 'loss_rpn_box_reg':
                    losses += loss_dict[key] * class_weights

            # Verifica se losses è un oggetto tensor
            optimizer.zero_grad()
            losses.backward() 
            optimizer.step()

            total_loss += losses.item()
            train_loop.set_postfix(loss=losses.item())

        losses_per_epoch.append(total_loss)
        print(f"Perdita Totale per l'epoca {epoch + 1}: {total_loss:.4f}")

        # Validazione
        model.eval()
        val_loop = tqdm(val_loader, desc="Validazione", leave=False)
        with torch.no_grad():
            for images, targets in val_loop:
                # Controlla se le immagini sono valide (non None)
                if images is None or targets is None:
                    continue
                
                images = [img.to(device) if img is not None else None for img in images]
                targets = [{k: v.to(device) if v is not None else None for k, v in t.items()} for t in targets]
                predictions = model(images)
                val_loop.set_postfix(processed=len(predictions))  # Placeholder per metriche future

        # Salva il modello
        if save_model:
            torch.save(model.state_dict(), f"model_epoch_{epoch + 1}.pth")
            print(f"Modello salvato: model_epoch_{epoch + 1}.pth")

    return losses_per_epoch

In [209]:
def test_model(model, test_loader, device):
    """
    Funzione per il testing del modello Faster R-CNN.
    
    Args:
    - model: il modello Faster R-CNN.
    - test_loader: DataLoader per il test set.
    - device: dispositivo su cui eseguire (es. 'cuda' o 'cpu').
    
    Returns:
    - predictions: lista delle predizioni per ogni batch (include 'boxes', 'labels', 'scores').
    """
    model.to(device)
    model.eval()
    predictions = []
    
    print("\nInizio testing...")
    test_loop = tqdm(test_loader, desc="Testing", leave=False)
    
    with torch.no_grad():
        for images, _ in test_loop:  # Durante il test, i target possono essere ignorati
            images = [img.to(device) for img in images]
            preds = model(images)
            
            # Predizioni di ciascun batch (contenente 'boxes', 'labels', 'scores')
            # Le predizioni sono in un formato di lista di dizionari
            for pred in preds:
                predictions.append({
                    'boxes': pred['boxes'].cpu().numpy(),
                    'labels': pred['labels'].cpu().numpy(),
                    'scores': pred['scores'].cpu().numpy()
                })
            
            # Aggiungi aggiornamenti su quante predizioni sono state processate
            test_loop.set_postfix(processed=len(predictions))

    print("Testing completato.")
    return predictions

In [199]:
# Carica il modello Faster R-CNN con ResNet50 e FPN
model = fasterrcnn_resnet50_fpn(pretrained=False)

num_classes = 12  # 11 classi + 1 per il background

# Modifica il numero di classi in output
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

# Imposta il dispositivo (GPU o CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Configurazione training
num_epochs = 2
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

In [202]:
# Calcola i pesi delle classi
class_weights = compute_class_weights(train_loader.dataset)

print(class_weights)

[5.00000000e+00 2.96286864e+00 1.93723199e+01 1.58589416e+02
 1.07289276e+02 1.24362542e+02 1.73934526e+00 4.47539333e+03
 3.31510617e+02 1.18963140e+02 1.59455819e+03 0.00000000e+00]


In [ ]:
# Esegui il training
losses_per_epoch = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    class_weights=class_weights,
    num_epochs=num_epochs,
    save_model=True
)


Epoca 1/2


Training:   1%|▏         | 211/15624 [01:22<1:45:08,  2.44it/s, loss=2.01] 

In [ ]:
# Testing
test_predictions = test_model(
    model=model,
    test_loader=test_loader_frcc,
    device=device
)